# Trajectory optimization

As a robot moves around the world, it can accumulates readings of its own motion. _"I have moved a meters forward!"_, a robot may exclaim after querying a sensor. _"One meter forward and twenty to the left!"_, after another reading. And you get how that goes.

After some time, a robot may have collected a _sequence_ of measurements that, when put together, are representative of its entire trajectory. However, there is an immediate danger in this formulation! If a single measurement is wrong, the rest will carry over its error.

For example, if we have the readings:
* `forward 1m and rotate 90 degree to the left`, 
* `forward 1m and rotate 90 degree to the left`, 
* `forward 1m and rotate 90 degree to the left`, 
* `forward 1m and rotate 90 degree to the left`, 

the robot believes that it drew a square and got back to its starting position. 

However, if the readings change to 

* `forward 1m and rotate 90 degree to the left`, 
* `forward 1m and rotate 90 degree to the left`, 
* `forward 1m and rotate` **45 degree to the left**, 
* `forward 1m and rotate 90 degree to the left`, 

then the robot believes that it traveled to a point to the left of its starting point. This is called odometry drift. It is the scenario where error in odometry readings get carried over to the next position estimates.

Hope is not all lost, however! Given additional information, robots can correct the estimates of their position in the world and also correct backwards (smooth) their entire trajectory. 
Back to our simple example, if the robot knew somehow that the end position of its third motion was one meter to the left of the origin, then it would get there corrected readings:

* `forward 1m and rotate 90 degree to the left`, 
* `forward 1m and rotate 90 degree to the left`, 
* ~~`forward 1m and rotate` 45 degree to the left~~  **at one meter to the left of the origin and facing it** 
* `forward 1m and rotate 90 degree to the left`, 

and the robot would believe that it is back at the initial position and also that it has turned 90 degrees in its third motion.

In this section we'll be exploring this problem, which we call **trajectory optimization**, in depth.

We'll be 
* Discussing rigid transformations in 2-dimensions.
* Discussing landmark observations.
* Discussing optimization problems in general.
* Formalizing the trajectory optimization problem.
* Solving the optimization problem with GTSAM.

Let's get to work!

# TODO add figure for odometry drift.

# Rigid Transformations in 2D

Notation
Transformations
Inverse transformations

Chaining together motion estimates.
Introducing the Trajectory (Graph) class.


### Implementing a transformation from human-readable inputs.

In [1]:
import numpy as np
import gtsam

In [ ]:
class Trajectory():
    def __init__(self) -> None:
        # Noise.
        self.ODOM_NOISE =     gtsam.noiseModel.Diagonal.Sigmas(np.array([0.3, 0.3, 0.1]))
        self.LANDMARK_NOISE = gtsam.noiseModel.Diagonal.Sigmas(np.array([0.03, 0.03, 0.03]))
        self.PRIOR_NOISE =    gtsam.noiseModel.Diagonal.Sigmas(np.array([0.00003, 0.00003, 0.00001]))

        # Landmarks.
        self.landmarks = set()

        # Measurements. Mapping nodes to RangeMeasurement's/
        self.measurements = {}
        
        # Keep track of xy edges and build matrix C only after optimizing for angles.
        self.edges = []

        # Should we optimize after every edge addition?
        self.online = False

### Inverse Transformations.

# Landmark observations.
What and why

Observation model. Adding observations to our class.
Relating pose estimates via landmark observations (loop closures)

### Creating virtual measurements between poses.

In [ ]:
# Given a dictionary relating odometry pose estimates to landmarks, and a relative pose between an odometry estimate and some landmark return a list of relative poses between the other nodes that have seen this landmark.

### Optimization Primer

In this section we are not going to dive too deep into the background of optimization theory. In fact, we will almost completely leave the actual optimization of the pose graph  (the trajectory optimization) as a black box.

It is helpful, however, to understand what optimization is and how it could be solved.

In [ ]:
# Least squares line fitting.


### Trajectory Optimization.

Formulation of cost function and analogy to least squares linear regression

provide code for optimization.